In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# SoSDoeScenario setUp

In [4]:
study_name = 'doe'
ns = f'{study_name}'
sc_name = "DoEEval"
#c_name = "SellarCoupling"
print(sc_name)

DoEEval


See §2 for Discipmline description

In [5]:
dspace_dict_eval = {'variable': ['x', 'z'],
                    'lower_bnd': [0., [-10., 0.]],
                    'upper_bnd': [10., [10., 10.]]}

In [6]:
dspace_eval = pd.DataFrame(dspace_dict_eval)
dspace_eval

,variable,lower_bnd,upper_bnd
0,x,0.0,10.0
1,z,"[-10.0, 0.0]","[10.0, 10.0]"


In [7]:
input_selection_x_z = {'selected_input': [False, True, False, False, True],
                       'full_name': ['DoEEval.Sellar_Problem.local_dv', 'x', 'y_1',
                                     'y_2',
                                     'z']}
input_selection_x_z = pd.DataFrame(input_selection_x_z)
input_selection_x_z 

,selected_input,full_name
0,False,DoEEval.Sellar_Problem.local_dv
1,True,x
2,False,y_1
3,False,y_2
4,True,z


In [8]:

output_selection_obj_y1_y2 = {'selected_output': [False, False, True, True, True],
                              'full_name': ['c_1', 'c_2', 'obj', 'y_1', 'y_2']}
output_selection_obj_y1_y2 = pd.DataFrame(output_selection_obj_y1_y2)
output_selection_obj_y1_y2

,selected_output,full_name
0,False,c_1
1,False,c_2
2,True,obj
3,True,y_1
4,True,y_2


In [9]:
repo = 'sos_trades_core.sos_processes.test'
proc_name = "test_sellar_doe_eval"

# usepydoe_lib

In [10]:
from gemseo.algos.doe.doe_factory import DOEFactory
pydoe_lib = DOEFactory()
for algo_name in pydoe_lib.algorithms:
  if algo_name not in ['ccdesign', 'CustomDOE', 'DiagonalDOE', 'OT_FACTORIAL', 'OT_COMPOSITE', 'OT_AXIAL',
                            'OT_OPT_LHS']:
       print(algo_name)
       algo = pydoe_lib.create(algo_name)
       samples = algo._generate_samples(n_samples=100, dimension=5)
       print(samples)

OT_SOBOL
[[0.5       0.5       0.5       0.5       0.5      ]
 [0.75      0.25      0.75      0.25      0.75     ]
 [0.25      0.75      0.25      0.75      0.25     ]
 [0.375     0.375     0.625     0.125     0.875    ]
 [0.875     0.875     0.125     0.625     0.375    ]
 [0.625     0.125     0.375     0.375     0.125    ]
 [0.125     0.625     0.875     0.875     0.625    ]
 [0.1875    0.3125    0.3125    0.6875    0.5625   ]
 [0.6875    0.8125    0.8125    0.1875    0.0625   ]
 [0.9375    0.0625    0.5625    0.9375    0.3125   ]
 [0.4375    0.5625    0.0625    0.4375    0.8125   ]
 [0.3125    0.1875    0.9375    0.5625    0.4375   ]
 [0.8125    0.6875    0.4375    0.0625    0.9375   ]
 [0.5625    0.4375    0.1875    0.8125    0.6875   ]
 [0.0625    0.9375    0.6875    0.3125    0.1875   ]
 [0.09375   0.46875   0.84375   0.40625   0.28125  ]
 [0.59375   0.96875   0.34375   0.90625   0.78125  ]
 [0.84375   0.21875   0.09375   0.15625   0.53125  ]
 [0.34375   0.71875   0.59375   0.656

# Doe scenario execution

# 1 Set Doe optim scenario lhs or fullfact

In [11]:
from gemseo.api import (
    configure_logger,
    get_algorithm_options_schema,
    get_available_doe_algorithms,
)
get_available_doe_algorithms()

['CustomDOE',
 'DiagonalDOE',
 'OT_SOBOL',
 'OT_RANDOM',
 'OT_HASELGROVE',
 'OT_REVERSE_HALTON',
 'OT_HALTON',
 'OT_FAURE',
 'OT_MONTE_CARLO',
 'OT_FACTORIAL',
 'OT_COMPOSITE',
 'OT_AXIAL',
 'OT_OPT_LHS',
 'OT_LHS',
 'OT_LHSC',
 'OT_FULLFACT',
 'OT_SOBOL_INDICES',
 'fullfact',
 'ff2n',
 'pbdesign',
 'bbdesign',
 'ccdesign',
 'lhs']

In [12]:
#my_doe_algo = "lhs"
my_doe_algo = "fullfact"

In [13]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.DoEEval is set to MDAJacobi
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


In [14]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	|_ DoEEval
		|_ Sellar_2
		|_ Sellar_1
		|_ Sellar_Problem


'Nodes representation for Treeview doe\n|_ doe\n\t|_ DoEEval\n\t\t|_ Sellar_2\n\t\t|_ Sellar_1\n\t\t|_ Sellar_Problem'

In [15]:
exec_eng.display_treeview_nodes(True)

INFO:SoS.EE:Nodes representation for Treeview doe
|_ doe
	-> acceleration
	-> authorize_self_coupled_disciplines
	-> cache_file_path
	-> cache_type
	-> chain_linearize
	-> debug_mode
	-> debug_mode_sellar
	-> epsilon0
	-> group_mda_disciplines
	-> linear_solver_MDA
	-> linear_solver_MDA_options
	-> linear_solver_MDA_preconditioner
	-> linear_solver_MDO
	-> linear_solver_MDO_options
	-> linear_solver_MDO_preconditioner
	-> linearization_mode
	-> max_mda_iter
	-> n_processes
	-> n_subcouplings_parallel
	-> relax_factor
	-> sub_mda_class
	-> tolerance
	-> tolerance_gs
	-> use_lu_fact
	-> warm_start
	-> warm_start_threshold
	-> x
	-> y_1
	-> y_2
	-> z
	<- c_1
	<- c_2
	<- obj
	<- residuals_history
	<- y_1_bis
	<- y_2_bis
	|_ DoEEval
		-> acceleration
		-> authorize_self_coupled_disciplines
		-> cache_file_path
		-> cache_type
		-> chain_linearize
		-> debug_mode
		-> epsilon0
		-> eval_inputs
		-> eval_outputs
		-> group_mda_disciplines
		-> linear_solver_MDA
		-> linear_solver_MDA_options


'Nodes representation for Treeview doe\n|_ doe\n\t-> acceleration\n\t-> authorize_self_coupled_disciplines\n\t-> cache_file_path\n\t-> cache_type\n\t-> chain_linearize\n\t-> debug_mode\n\t-> debug_mode_sellar\n\t-> epsilon0\n\t-> group_mda_disciplines\n\t-> linear_solver_MDA\n\t-> linear_solver_MDA_options\n\t-> linear_solver_MDA_preconditioner\n\t-> linear_solver_MDO\n\t-> linear_solver_MDO_options\n\t-> linear_solver_MDO_preconditioner\n\t-> linearization_mode\n\t-> max_mda_iter\n\t-> n_processes\n\t-> n_subcouplings_parallel\n\t-> relax_factor\n\t-> sub_mda_class\n\t-> tolerance\n\t-> tolerance_gs\n\t-> use_lu_fact\n\t-> warm_start\n\t-> warm_start_threshold\n\t-> x\n\t-> y_1\n\t-> y_2\n\t-> z\n\t<- c_1\n\t<- c_2\n\t<- obj\n\t<- residuals_history\n\t<- y_1_bis\n\t<- y_2_bis\n\t|_ DoEEval\n\t\t-> acceleration\n\t\t-> authorize_self_coupled_disciplines\n\t\t-> cache_file_path\n\t\t-> cache_type\n\t\t-> chain_linearize\n\t\t-> debug_mode\n\t\t-> epsilon0\n\t\t-> eval_inputs\n\t\t-> eva

In [16]:
exec_eng.dm.get_data('doe.sub_mda_class')['value']

'MDAJacobi'

Possible algo options:

In [17]:
from gemseo.algos.doe.doe_factory import DOEFactory
driver_lib = DOEFactory().create(my_doe_algo)
driver_lib.init_options_grammar(my_doe_algo)
algo_options_keys = list(driver_lib.opt_grammar.properties_dict.keys())
print(algo_options_keys)

['n_samples', 'levels', 'eval_jac', 'n_processes', 'wait_time_between_samples', 'max_time']


In [18]:
schema_dict = driver_lib.opt_grammar.schema.to_dict()
properties = schema_dict.get(driver_lib.opt_grammar.PROPERTIES_FIELD)
algo_options_keys = list(properties.keys()) 
algo_options_keys

['n_samples',
 'levels',
 'eval_jac',
 'n_processes',
 'wait_time_between_samples',
 'max_time']

In [19]:
#-- set up disciplines in Scenario
disc_dict = {}
# Doe inputs
# 'lhs', 'CustomDOE', 'fullfact', ...
n_samples = 100
disc_dict[f'{ns}.DoEEval.sampling_algo'] = "fullfact"
disc_dict[f'{ns}.DoEEval.design_space'] = dspace_eval
disc_dict[f'{ns}.DoEEval.algo_options'] = {'n_samples': n_samples, 'fake_option': 'fake_option'}
disc_dict[f'{ns}.DoEEval.eval_inputs'] = input_selection_x_z
disc_dict[f'{ns}.DoEEval.eval_outputs'] = output_selection_obj_y1_y2


disc_dict

{'doe.DoEEval.sampling_algo': 'fullfact',
 'doe.DoEEval.design_space':   variable     lower_bnd     upper_bnd
 0        x           0.0          10.0
 1        z  [-10.0, 0.0]  [10.0, 10.0],
 'doe.DoEEval.algo_options': {'n_samples': 100, 'fake_option': 'fake_option'},
 'doe.DoEEval.eval_inputs':    selected_input                        full_name
 0           False  DoEEval.Sellar_Problem.local_dv
 1            True                                x
 2           False                              y_1
 3           False                              y_2
 4            True                                z,
 'doe.DoEEval.eval_outputs':    selected_output full_name
 0            False       c_1
 1            False       c_2
 2             True       obj
 3             True       y_1
 4             True       y_2}

Remark: in fact it is a <b> DoE optim </b> (with 'obj' variable taken as obj function) ! Here dspace variable was used.

In [20]:
#my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(disc_dict,orient='index')

,0
doe.DoEEval.sampling_algo,fullfact
doe.DoEEval.design_space,variable lower_bnd upper_bnd 0 ...
doe.DoEEval.algo_options,"{'n_samples': 100, 'fake_option': 'fake_option'}"
doe.DoEEval.eval_inputs,selected_input full_...
doe.DoEEval.eval_outputs,selected_output full_name 0 Fals...


In [21]:
#exec_eng.dm.set_values_from_dict(disc_dict) #KO!
exec_eng.load_study_from_input_dict(disc_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.DoEEval is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe.DoEEval is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
ERROR:SoS.EE.DataManager:Variable: doe.x value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.z value is not set!
ERROR:SoS.EE.DataManager:Variable: doe.DoEEval.Sellar_Problem.local_dv value is not set!


In [22]:
eval_inputs=input_selection_x_z;
eval_inputs['selected_input']==True

0    False
1     True
2    False
3    False
4     True
Name: selected_input, dtype: bool

# 2 Display Disciplines I/O variables

In [24]:
sellar1 = exec_eng.dm.get_disciplines_with_name("doe." + "DoEEval.Sellar_1")[0]

In [25]:
type(sellar1)

sos_trades_core.sos_wrapping.test_discs.sellar.Sellar1

In [26]:
DESC_IN = sellar1.DESC_IN
DESC_OUT = sellar1.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [27]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar


In [28]:
DESC_OUT_df

,type,visibility,namespace
y_1,float,Shared,ns_OptimSellar
y_1_bis,float,Shared,ns_OptimSellar


In [29]:
sellar2 = exec_eng.dm.get_disciplines_with_name("doe." + "DoEEval.Sellar_2")[0]

In [30]:
DESC_IN = sellar2.DESC_IN
DESC_OUT = sellar2.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [31]:
DESC_IN_df

,type,visibility,namespace,default
y_1,float,Shared,ns_OptimSellar,NaN
z,array,Shared,ns_OptimSellar,NaN
debug_mode_sellar,bool,Shared,ns_OptimSellar,False


In [32]:
DESC_OUT_df

,type,visibility,namespace
y_2,float,Shared,ns_OptimSellar
y_2_bis,float,Shared,ns_OptimSellar


In [33]:
sellarP = exec_eng.dm.get_disciplines_with_name("doe." + "DoEEval.Sellar_Problem")[0]

In [34]:
DESC_IN = sellarP.DESC_IN
DESC_OUT = sellarP.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [35]:
DESC_IN_df

,type,visibility,namespace
x,float,Shared,ns_OptimSellar
y_1,float,Shared,ns_OptimSellar
y_2,float,Shared,ns_OptimSellar
z,array,Shared,ns_OptimSellar
local_dv,float,NaN,NaN


In [36]:
DESC_OUT_df

,type,visibility,namespace
c_1,array,Shared,ns_OptimSellar
c_2,array,Shared,ns_OptimSellar
obj,array,Shared,ns_OptimSellar


# 3 Provide Discipline inputs

In [37]:
# Sellar inputs
local_dv = 10.
values_dict = {}
values_dict[f'{ns}.x'] = 1.  
values_dict[f'{ns}.y_1'] = 1. 
values_dict[f'{ns}.y_2'] = 1. 
values_dict[f'{ns}.z'] = array([1., 1.])
values_dict[f'{ns}.{sc_name}.Sellar_Problem.local_dv'] = local_dv
#values_dict[f'{ns}.cache_type'] = 'MemoryFullCache'  # KO !!!
#values_dict[f'{ns}.{sc_name}.cache_type'] = 'MemoryFullCache' #OK but remain SimpleCache
#values_dict[f'{ns}.{sc_name}.Sellar_1.cache_type'] = 'MemoryFullCache'
#values_dict[f'{ns}.{sc_name}.Sellar_2.cache_type'] = 'MemoryFullCache'
#values_dict[f'{ns}.{sc_name}.Sellar_Problem.cache_type'] = 'MemoryFullCache'  # KO !!! Problem with local_dv (the variable that is not shared)

values_dict
#my_dict = exec_eng.get_anonimated_data_dict()
pd.DataFrame.from_dict(values_dict,orient='index')

,0
doe.x,1.0
doe.y_1,1.0
doe.y_2,1.0
doe.z,"[1.0, 1.0]"
doe.DoEEval.Sellar_Problem.local_dv,10.0


In [38]:
#exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.load_study_from_input_dict(values_dict)
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi


# 4 Run Doe scenario

In [39]:
res = exec_eng.execute()
# res is without interest (return not the expected values but probably default values ) !

INFO:SoS.EE:PROCESS EXECUTION doe STARTS...
INFO:SoS.EE.DOE:{'n_samples': 100, 'fake_option': 'fake_option'}
INFO:gemseo.algos.doe.doe_lib:Full factorial design required. Number of samples along each direction for a design vector of size 3 with 100 samples: 4
INFO:gemseo.algos.doe.doe_lib:Final number of samples for DOE = 64 vs 100 requested
INFO:SoS.EE.DOE:running sos eval in sequential
  0%|                                                                        | 0/64 [00:00<?, ?it/s]INFO:SoS.EE.DOE:   Scenario_1 is running.
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	7.021909e+07
INFO:gemseo.mda.mda:	2	6.512296e+06
INFO:gemseo.mda.mda:	3	1.278903e+06
INFO:gemseo.mda.mda:	4	5.051838e+04
INFO:gemseo.mda.mda:	5	2.537783e+00
INFO:gemseo.mda.mda:	6	8.882373e-07
  2%|█                                                            

INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	5.598170e+07
INFO:gemseo.mda.mda:	2	4.001377e+06
INFO:gemseo.mda.mda:	3	7.804442e+05
INFO:gemseo.mda.mda:	4	1.443288e+04
INFO:gemseo.mda.mda:	5	1.497898e+00
INFO:gemseo.mda.mda:	6	1.610912e-07
 20%|████████████▊                                                  | 13/64 [00:07<00:27,  1.82it/s]INFO:SoS.EE.DOE:   Scenario_14 is running.
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	1.192305e+05
INFO:gemseo.mda.mda:	3	2.378719e+04
INFO:gemseo.mda.mda:	4	1.000020e+01
INFO:gemseo.mda.mda:	5	8.345355e-06
INFO:gemseo.mda.mda:	6	0.000000e+00
 22%|█████████████▊                                                 | 14/64 [00:07<

INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	3.147030e+05
INFO:gemseo.mda.mda:	3	6.208339e+04
INFO:gemseo.mda.mda:	4	4.484315e+02
INFO:gemseo.mda.mda:	5	1.528705e-02
INFO:gemseo.mda.mda:	6	0.000000e+00
 41%|█████████████████████████▌                                     | 26/64 [00:14<00:20,  1.83it/s]INFO:SoS.EE.DOE:   Scenario_27 is running.
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	2.835087e+05
INFO:gemseo.mda.mda:	3	5.603914e+04
INFO:gemseo.mda.mda:	4	2.998464e+02
INFO:gemseo.mda.mda:	5	6.971874e-03
INFO:gemseo.mda.mda:	6	0.000000e+00
 42%|██████████████████████████▌                                    | 27/64 [00:14<

INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	2.559937e+05
INFO:gemseo.mda.mda:	3	5.068977e+04
INFO:gemseo.mda.mda:	4	2.018395e+02
INFO:gemseo.mda.mda:	5	3.210153e-03
INFO:gemseo.mda.mda:	6	1.256074e-09
 61%|██████████████████████████████████████▍                        | 39/64 [00:21<00:13,  1.82it/s]INFO:SoS.EE.DOE:   Scenario_40 is running.
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	2.382869e+05
INFO:gemseo.mda.mda:	3	4.723679e+04
INFO:gemseo.mda.mda:	4	1.527004e+02
INFO:gemseo.mda.mda:	5	1.855005e-03
INFO:gemseo.mda.mda:	6	0.000000e+00
 62%|███████████████████████████████████████▍                       | 40/64 [00:21<00:13,  1.82it/s]INFO:SoS.EE.DOE:   Scenario_41 is running.
INFO:SoS.EE:conf

INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	2.357023e+06
INFO:gemseo.mda.mda:	2	1.093339e+05
INFO:gemseo.mda.mda:	3	2.182110e+04
INFO:gemseo.mda.mda:	4	7.092007e+00
INFO:gemseo.mda.mda:	5	4.207848e-06
INFO:gemseo.mda.mda:	6	1.256074e-09
 81%|███████████████████████████████████████████████████▏           | 52/64 [00:28<00:06,  1.82it/s]INFO:SoS.EE.DOE:   Scenario_53 is running.
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling doe is set to MDAJacobi
INFO:SoS.EE.Coupling:doe.DoEEval MDA history
INFO:SoS.EE.Coupling:	It.	Res. norm
INFO:gemseo.mda.mda:	1	7.008372e+07
INFO:gemseo.mda.mda:	2	4.693892e+06
INFO:gemseo.mda.mda:	3	9.325130e+05
INFO:gemseo.mda.mda:	4	4.750815e+04
INFO:gemseo.mda.mda:	5	1.939434e+01
INFO:gemseo.mda.mda:	6	7.569479e-05
INFO:gemseo.mda.mda:	7	2.512148e-09
 83%|████████████████████████████████████████████████████▏          | 53/64 [00:29<00:06,  1.82it/s]INFO:SoS.EE.DOE:   Scenario_54 is running.
INFO:SoS.EE:configuring .

# 5 Display Doe  Scenario results

In [40]:
# retrieve discipline to check the result...
doe_disc = exec_eng.dm.get_disciplines_with_name("doe." + sc_name)[0]

In [41]:
type(doe_disc)

sos_trades_core.sos_wrapping.analysis_discs.doe_eval.DoeEval

In [42]:
doe_disc.get_disc_full_name()

'doe.DoEEval'

In [43]:
doe_disc_input = doe_disc.get_sosdisc_inputs()
pd.DataFrame.from_dict(doe_disc_input,orient='index')

,0
sampling_algo,fullfact
eval_inputs,selected_input full_...
eval_outputs,selected_output full_name 0 Fals...
n_processes,1
wait_time_between_fork,0.0
linearization_mode,auto
cache_type,SimpleCache
cache_file_path,None
debug_mode,
design_space,variable lower_bnd upper_bnd 0 ...


In [45]:
#doe_disc_output = doe_disc.get_sosdisc_outputs()
doe_disc_output = doe_disc.get_sosdisc_outputs('doe_samples_dataframe')

In [46]:
doe_disc_output

,scenario,x,z
0,scenario_1,0.000000,"[-10.0, 0.0]"
1,scenario_2,3.333333,"[-10.0, 0.0]"
2,scenario_3,6.666667,"[-10.0, 0.0]"
3,scenario_4,10.000000,"[-10.0, 0.0]"
4,scenario_5,0.000000,"[-3.333333333333334, 0.0]"
...,...,...,...
59,scenario_60,10.000000,"[3.333333333333332, 10.0]"
60,scenario_61,0.000000,"[10.0, 10.0]"
61,scenario_62,3.333333,"[10.0, 10.0]"
62,scenario_63,6.666667,"[10.0, 10.0]"


In [49]:
dimension = sum([len(sublist) if isinstance(
    sublist, list) else 1 for sublist in list(dspace_eval['variable'].values)])
dimension

2

In [50]:
theoretical_fullfact_levels = int(n_samples ** (1.0 / dimension))
theoretical_fullfact_levels

10

In [51]:
theoretical_fullfact_samples = theoretical_fullfact_levels ** dimension
theoretical_fullfact_samples

100

In [52]:
len(doe_disc_output)

64